In [ ]:
from donkeycar.parts.keras import KerasPilot

from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Convolution2D, Convolution2D, MaxPooling2D, Reshape, BatchNormalization
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Cropping2D, Lambda

In [ ]:
# Croppa vrh slike zbog horizont problema
def adjust_input_shape(input_shape, roi_crop):
    height = input_shape[0]
    new_height = height - roi_crop[0] - roi_crop[1]
    return (new_height, input_shape[1], input_shape[2])

# Ovdje stavi definiciju svojeg modela
# Ako koristiš samo built-in modele od Donkeya, nađi definiciju u donkeycar/parts/keras.py
# Also, dodijeli svakom konvolucijskom sloju ime "convx" (x ∈ ℕ)

def customModel(num_outputs=2, input_shape=(640,480,3), roi_crop=(0,0)):

    input_shape = adjust_input_shape(input_shape, roi_crop)
    img_in = Input(shape=input_shape, name='img_in')
    x = img_in
    
    # Dropout rate
    keep_prob = 0.5
    rate = 1 - keep_prob
    
    # Convolutional Layer 1
    x = Convolution2D(filters=12, kernel_size=5, strides=(2, 2), activation='relu', name="conv1")(x)
    x = Dropout(rate)(x)

    # Convolutional Layer 2
    x = Convolution2D(filters=24, kernel_size=5, strides=(2, 2), activation='relu', name="conv2")(x)
    x = Dropout(rate)(x)

    # Convolutional Layer 3
    x = Convolution2D(filters=48, kernel_size=5, strides=(2, 2), activation='relu', name="conv3")(x)
    x = Dropout(rate)(x)

    # Convolutional Layer 4
    x = Convolution2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu', name="conv4")(x)
    x = Dropout(rate)(x)

    # Convolutional Layer 5
    x = Convolution2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu', name="conv5")(x)
    x = Dropout(rate)(x)

    # Flatten Layers
    x = Flatten()(x)

    # Fully Connected Layer 1
    x = Dense(100, activation='relu')(x)

    # Fully Connected Layer 2
    x = Dense(50, activation='relu')(x)

    # Fully Connected Layer 3
    x = Dense(25, activation='relu')(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(5, activation='relu')(x)
    outputs = []
    
    for i in range(num_outputs):
        outputs.append(Dense(1, activation='linear', name='n_outputs' + str(i))(x))
        
    model = Model(inputs=[img_in], outputs=outputs)
    
    return model

In [ ]:
model = customModel()
# Ovdje mu proslijedi path do spremljenih weightova/treniranog modela, .h5 file
model.load_weights('test.h5')

In [ ]:
img_in = Input(shape=(640,480,3), name='img_in')

x = img_in
x = Convolution2D(filters=12, kernel_size=5, strides=(2, 2), activation='relu', name="conv1")(x)
x = Convolution2D(filters=24, kernel_size=5, strides=(2, 2), activation='relu', name="conv2")(x)
x = Convolution2D(filters=48, kernel_size=5, strides=(2, 2), activation='relu', name="conv3")(x)
x = Convolution2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu', name="conv4")(x)
zadnjiConv = Convolution2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu', name="conv5")(x)  

# Napiši koliko conv layera imaš
brojConvSlojeva = 5
convolution_part = Model(inputs=[img_in], outputs=[zadnjiConv])
for layer_num in range(1, brojConvSlojeva):
    convolution_part.get_layer('conv' + str(layer_num)).set_weights(model.get_layer('conv' + str(layer_num)).get_weights())

In [ ]:
from tensorflow.python.keras import backend as K

inp = convolution_part.input                                           # input placeholder
outputs = [layer.output for layer in convolution_part.layers[1:]]          # all layer outputs
functor = K.function([inp], outputs)

In [ ]:
import tensorflow as tf
import numpy as np
import pdb

# 3x3 kernel jedinice
kernel_3x3 = tf.constant(np.array([
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]]
]), tf.float32)

# 5x5 kernel jedinice
kernel_5x5 = tf.constant(np.array([
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]]
]), tf.float32)

# Ovdje stavi kernele koje koristiš u kojem sloju konvolucije, u mene su prva tri 5x5, zadnja dva 3x3
layers_kernels = {5: kernel_3x3, 4: kernel_3x3, 3: kernel_5x5, 2: kernel_5x5, 1: kernel_5x5}

# Ovdje stavi strideove koje koristiš [p, s, s, p], prva i zadnja jedinica su padding afaik
layers_strides = {5: [1, 1, 1, 1], 4: [1, 1, 1, 1], 3: [1, 2, 2, 1], 2: [1, 2, 2, 1], 1: [1, 2, 2, 1]}

In [ ]:
# forward prop i vizualizacijska maska
def compute_visualisation_mask(img):
    activations = functor([np.array([img])])
    activations = [np.reshape(img, (1, img.shape[0], img.shape[1], img.shape[2]))] + activations
    upscaled_activation = np.ones((53, 73))
    for layer in [5, 4, 3, 2, 1]:
        averaged_activation = np.mean(activations[layer], axis=3).squeeze(axis=0) * upscaled_activation
        output_shape = (activations[layer - 1].shape[1], activations[layer - 1].shape[2])
        x = tf.constant(
            np.reshape(averaged_activation, (1,averaged_activation.shape[0],averaged_activation.shape[1],1)),
            tf.float32
        )
        conv = tf.nn.conv2d_transpose(
            x, layers_kernels[layer],
            output_shape=(1,output_shape[0],output_shape[1], 1), 
            strides=layers_strides[layer], 
            padding='VALID'
        )
        with tf.Session() as session:
            result = session.run(conv)
        upscaled_activation = np.reshape(result, output_shape)
    final_visualisation_mask = upscaled_activation
    return (final_visualisation_mask - np.min(final_visualisation_mask))/(np.max(final_visualisation_mask) - np.min(final_visualisation_mask))

In [ ]:
import cv2
import numpy as np

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

def save_movie_mp4(image_array, filename='output.mp4', fps=30):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)
    
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps, metadata=dict(artist='Me'), bitrate=1800)
    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    anim.save(filename, writer=writer)
    display(HTML(anim.to_html5_video()))

In [ ]:
from glob import iglob

In [ ]:
# Putanja do slika/dataseta
pathToData = 'data/imgs/'
# Output video naziv
output = "saliency.mp4"
# Output FPS
fps = 30

imgs = []
alpha = 0.004
beta = 1.0 - alpha
counter = 0
for path in sorted(iglob(pathToData + '*.jpg')):
    img = cv2.imread(path)
    salient_mask = compute_visualisation_mask(img)
    salient_mask_stacked = np.dstack((salient_mask,salient_mask))
    salient_mask_stacked = np.dstack((salient_mask_stacked,salient_mask))
    blend = cv2.addWeighted(img.astype('float32'), alpha, salient_mask_stacked, beta, 0.0)
    imgs.append(blend)
    counter += 1
    if counter >= 400:
        break

In [ ]:
plot_movie_mp4(imgs, output, fps)